# Filtrado colaborativo (preprocesamiento)

**Autor**: Arturo Sánchez Palacio

Basado en: https://github.com/lazyprogrammer

**Fecha de última revisión: 15/I/2020**

Estos son los módulos que usaremos en esta tarea:

__Nota.__ Si no se dispone de alguno de los módulos se puede instalar desde Jupyter mediante la instrucción:

dependiendo de si se emplea pip o Anaconda. El módulo debe ser instalado antes de ser importado.

In [2]:
import pandas as pd
import pickle

Comenzamos leyendo los datos del archivo csv `rating.csv`:

In [3]:
df = pd.read_csv('./data/rating.csv')

Comprobamos el tamaño del archivo y que se haya leído correctamente:

In [4]:
df.shape

(20000263, 4)

In [5]:
df.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [6]:
df.tail()

,userId,movieId,rating,timestamp
20000258,138493,68954,4.5,2009-11-13 15:42:00
20000259,138493,69526,4.5,2009-12-03 18:31:48
20000260,138493,69644,3.0,2009-12-07 18:10:57
20000261,138493,70286,5.0,2009-11-13 15:42:24
20000262,138493,71619,2.5,2009-10-17 20:25:36


Parece que se ha cargado correctamente.

Nuestra primera tarea es construir la matriz de usuarios-valoraciones.

Cuando trabajamos con datos y especialmente con grandes cantidades de datos siempre debemos tener en mente la eficiencia computacional. Así a la hora de construir la matriz nos interesa que los índices estén aprovechados, es decir:

In [7]:
print(df.userId.min(), df.userId.max())

1 138493


Que los usuarios estén identificados por índices consecutivos sin huecos. Estos huecos supondrían filas vacías en nuestra matriz que ocuparían memoria sin aportar ninguna información ya que el índice es simplemente informativo.

In [8]:
df.userId.nunique()

138493

Vemos que hay el mismo número de usuarios distintos que el máximo índice luego no hay huecos. Como la indexación en Python comienza en 0 restamos uno a los índices:

In [9]:
df.userId = df.userId - 1

In [10]:
print(df.userId.min(), df.userId.max())

0 138492


Realizamos la misma comprobación para los índices asociados a las películas:

In [11]:
print(df.movieId.min(), df.movieId.max())

1 131262


In [12]:
df.movieId.nunique()

26744

En este caso los índices no están asignados secuencialmente (puede que se hayan descartado películas sin votos, por ejemplo). Realizamos una reindexación de las películas para tenerlas identificadas de manera secuencial:

In [13]:
unique_movie_ids = set(df.movieId.values) #cogemos los índices únicos
diccionario_indice_peli = {} #iniciamos un diccionario vacío en que la clave sera la película y el valor el nuevo índice
nuevo_indice = 0 #empezamos la indexación en cero
for pelicula in unique_movie_ids: #para cada película
    diccionario_indice_peli[pelicula] = nuevo_indice #asociamos a la película un nuevo índice
    nuevo_indice += 1 #avanzamos uno en la numeración 
    
df['movie_idx'] = df.apply(lambda row: diccionario_indice_peli[row.movieId], axis=1) # Generamos una nueva columna con los nuevos índices (ahora sí empezando en 0 y secuenciales)

__Nota.__ Estamos añadiendo 20 millones de datos así que es normal que tarde un rato.

Comprobamos que la columna se ha añadido al dataframe y que todo es correcto:

In [14]:
df.head()

,userId,movieId,rating,timestamp,movie_idx
0,0,2,3.5,2005-04-02 23:53:47,2
1,0,29,3.5,2005-04-02 23:31:16,29
2,0,32,3.5,2005-04-02 23:33:39,32
3,0,47,3.5,2005-04-02 23:32:07,47
4,0,50,3.5,2005-04-02 23:29:40,50


In [15]:
print(df.movie_idx.min(), df.movie_idx.max())

0 26743


Finalmente en este ejercicio no usaremos la fecha así que podemos eliminar la columna para ahorrar memoria:

In [16]:
df = df.drop(columns=['timestamp'])

In [17]:
df.head()

,userId,movieId,rating,movie_idx
0,0,2,3.5,2
1,0,29,3.5,29
2,0,32,3.5,32
3,0,47,3.5,47
4,0,50,3.5,50


Tras realizar todas las transformaciones guardamos los datos en un nuevo archivo para no perder tiempo en realizar los cambios en un futuro:

In [18]:
df.to_csv('./data/preprocessed_rating.csv', index=False)

Hasta aquí nuestro primer preprocesamiento de datos.

Este método no es escalable. Si se tienen conocimientos de Spark la implementación se podría hacer de manera paralelizada pero como dichos conocimientos no son requisitos para este curso lo que haremos será reducir nuestros datos seleccionando un subconjuntos de usuarios y películas. Como discutimos previamente en la teoría la idea más interesante es elegir las películas que más veces han sido votadas y los usuarios que más veces han votado.

Como ahora nuestros índices son secuenciales podemos afirmar que:

In [19]:
M = df.movie_idx.max() + 1 #número de películas
N = df.userId.max() + 1 #número de usuarios

La función `Counter` del módulo `collections` nos permite hacer un conteo eficiente de valores agrupados devolviéndonos un objeto counter (similar a un diccionario con clave el índice y valor el número de apariciones). 

__Nota.__ El módulo `collections` viene automáticamente instalado.

In [20]:
from collections import Counter

Ejemplo:

In [21]:
Counter(df.movie_idx)[10] #veces que la película 0 ha sido votada

29005

Contamos pues los votos emitidos por cada usuario y los votos sobre cada película respectivamente:

In [22]:
user_ids_count = Counter(df.userId)
movie_ids_count = Counter(df.movie_idx)

Ahora fijamos el tamaño de muestra que vamos a elegir. En un proyecto real estos valores no son dados ni elegidos al azar. Se realizan diferentes pruebas y se comparan los resultados. En esta primera aproximación simplemente elegiremos valores que permitan un fácil manipulación de los datos:

In [23]:
m = 1000 # número de películas elegidas
n = 2500 # número de usuarios elegidos

`Counter` incluye el método `most_common()` que nos permite extraer los índices asociados a los x valores más frecuentes:

In [24]:
user_ids = [u for u, c in user_ids_count.most_common(n)] # para cada índice y valor en el counter extraigo el índice
movie_ids = [m for m, c in movie_ids_count.most_common(m)]

Una vez elegidos los índices de las películas y usuarios con los que trabajaremos extraemos todos los votos de dichos usuarios sobre dichas películas de todos los datos iniciales, rechazando todas aquellas valoraciones realizadas por usuarios no seleccionados o sobre películas no seleccionadas:

In [25]:
df_small = df[df.userId.isin(user_ids) & df.movie_idx.isin(movie_ids)].copy()

__Nota.__ El método `isin()` simplemente indica si un elemento está dentro de otro devolviendo un booleano.

__PROBLEMA.__ Ahora que hemos extraído ciertos usuarios/película y rechazado otros los índices vuelven a no ser secuenciales. Aplicamos de nuevo el mismo proceso

__Nota.__ Obviamente ahora nos damos cuenta de que realizar esto al principio era innecesario pero este ejemplo es didáctico y busca en parte reproducir las vicisitudes al trabajar con datos.

Creamos los diccionarios:

In [26]:
nuevo_user_id_map = {}
i = 0
for antiguo in user_ids:
    nuevo_user_id_map[antiguo] = i
    i += 1
    
nuevo_movie_id_map = {}
j = 0
for antiguo in movie_ids:
    nuevo_movie_id_map[antiguo] = j
    j += 1

Aplicamos la transformación a las columnas adecuadas:

In [27]:
df_small.loc[:, 'userId'] = df_small.apply(lambda row: nuevo_user_id_map[row.userId], axis=1)
df_small.loc[:, 'movie_idx'] = df_small.apply(lambda row: nuevo_movie_id_map[row.movie_idx], axis=1)

__Nota.__ De nuevo esto llevará un tiempo.

Una vez disponemos de los datos adecuados comprobamos que la tabla se ha guardado de forma correcta y de nuevo almacenamos los datos en un csv:

In [28]:
df_small.head()

,userId,movieId,rating,movie_idx
11454,1900,14,1.0,828
11455,1900,17,3.0,148
11456,1900,21,3.0,103
11457,1900,25,4.0,136
11459,1900,29,3.0,564


In [29]:
df_small.to_csv('./data/reduced_rating.csv', index=False)

Estos son los datos finales con los que trabajaremos, sin embargo, no es su estructura óptima. Pandas es una estructura muy eficiente pero pesada en ciertos sentidos. En este caso nos interesa poder almacenar los datos en __diccionarios__.

Iterar sobre la matriz tendría coste O(NM) mientras que iterar sobre diccionarios tiene un coste O(|$\Omega$|) con |$\Omega$| el número total de valoraciones. 

Procedemos pues a almacenar la información en diccionarios:

In [30]:
M = m #los valores son los elegidos manualmente antes
N = n

Dividimos los datos en entrenamiento (80%) y test (20%):

In [31]:
from sklearn.utils import shuffle

In [32]:
df_small = shuffle(df_small) #mezclamos los datos aleatoriamente
cutoff = int(0.8*len(df_small)) #fijamos un umbral del 80%
df_train = df_small.iloc[:cutoff]  #hasta el 80% datos de entrenamiento
df_test = df_small.iloc[cutoff:] #hasta el 20% datos de validación

Vamos a construir tres diccionarios:

* `usuario_pelicula`: indica qué usuarios han valorado qué películas.
* `pelicula_usuario`: indica qué películas han sido valoradas por qué usuarios.
* `usuariopeli_rating`: indica dada una tupla de usuario y película como clave cual ha sido su valoración.

Inicializamos los diccionarios vacíos:

In [33]:
usuario_pelicula = {}
pelicula_usuario = {}
usuariopeli_rating = {}

Vamos a definir una función para construir nuestros diccionarios de entrenamiento:

In [34]:
count = 0
def crear_usuario_pelicula_y_pelicula_usuario(row):
    global count
    count += 1
    if count % 100000 == 0:
        print("Procesado: %.3f" % (float(count)/cutoff)) #pequeño truco para ver a que velocidad se procesan los datos

    i = int(row.userId) # i es el usuario
    j = int(row.movie_idx) #j es la película
    
    #Diccionario usuario_pelicula
    if i not in usuario_pelicula:
        usuario_pelicula[i] = [j] # si el usuario i no está en el diccionario lo añadimos con valor la película j
    else:
        usuario_pelicula[i].append(j) # si i ya está en el diccionario le añadimos al valor la nueva película
        
    # Diccionario pelicula_usuario
    if j not in pelicula_usuario:
        pelicula_usuario[j] = [i] # si la peli j no está en el diccionario la añadimos con valor el usuario i
    else:
        pelicula_usuario[j].append(i) # si j ya está en el diccionario el añadimos al valor el nuevo ususario
    
    # Diccionario usuariopeli_rating
    usuariopeli_rating[(i,j)] = row.rating # añado la tupla (usuario, pelicula) como clave y la valoración como valor

Aplico la función a los datos de entrenamiento:

In [35]:
df_train.apply(crear_usuario_pelicula_y_pelicula_usuario, axis=1)

Procesado: 0.091
Procesado: 0.183
Procesado: 0.274
Procesado: 0.366
Procesado: 0.457
Procesado: 0.549
Procesado: 0.640
Procesado: 0.732
Procesado: 0.823
Procesado: 0.915


13517842    None
9894285     None
17836332    None
16964913    None
7535836     None
1887897     None
15008012    None
9775374     None
18980394    None
1466943     None
9871893     None
18230181    None
18638500    None
11169012    None
1895354     None
18371244    None
7576142     None
18596775    None
4049501     None
19742337    None
1594351     None
2191027     None
11679914    None
15034842    None
11256597    None
13164090    None
5098191     None
9781971     None
1404206     None
4268109     None
            ... 
19475347    None
14535130    None
6638533     None
19153159    None
12241086    None
1846677     None
18863955    None
13518258    None
3806947     None
6153023     None
19887636    None
13732391    None
13947048    None
11143912    None
6448777     None
19978691    None
18413119    None
14938646    None
14803849    None
538320      None
14378378    None
9846839     None
7148087     None
8163133     None
8599245     None
11306377    None
9097550     None
10636885    No

Para validación solo necesitaremos un diccionario de `usuariopeli_rating` así que definimos una función que construya únicamente dicho diccionario:

In [36]:
usuariopeli_rating_test = {}
count = 0
def crea_usuariopeli_rating_test(row):
    global count
    count += 1
    if count % 100000 == 0:
        print("Procesado: %.3f" % (float(count)/len(df_test)))

    i = int(row.userId)
    j = int(row.movie_idx)
    usuariopeli_rating_test[(i,j)] = row.rating

Aplicamos la función:

In [37]:
df_test.apply(crea_usuariopeli_rating_test, axis=1)

Procesado: 0.366
Procesado: 0.732


488896      None
2513579     None
15988222    None
17695168    None
6098887     None
515365      None
15775392    None
3359156     None
2282403     None
2666421     None
19978251    None
5981394     None
16433352    None
15065359    None
1535322     None
7061156     None
15108163    None
14562364    None
14038287    None
13193430    None
7461338     None
12151619    None
7287537     None
18282324    None
3424915     None
11022240    None
4039671     None
5972466     None
14494503    None
9698590     None
            ... 
11147724    None
10051940    None
7326571     None
13482374    None
7612147     None
7626165     None
2907733     None
14637323    None
16562311    None
8846014     None
18547212    None
13982444    None
5019478     None
11653411    None
18415495    None
15876407    None
15360367    None
10670590    None
18412908    None
10674592    None
1547269     None
16429073    None
15928059    None
6982752     None
19847214    None
18212389    None
2360925     None
18934562    No

Ya hemos construido nuestros cuatro diccionarios. Finalmente los almacenamos en sendos archivos .json:

__Nota.__ `pickle` puede guardar cualquier tipo de objeto de Python.
Técnicamente son archivos binarios y no json porque la clave es un string y en los json es un entero. `pickle` es parte de Python y no requiere instalación:

In [38]:
import pickle
with open('./data/usuario_pelicula.json', 'wb') as f:
    pickle.dump(usuario_pelicula, f)

with open('./data/pelicula_usuario.json', 'wb') as f:
    pickle.dump(pelicula_usuario, f)

with open('./data/usuariopeli_rating.json', 'wb') as f:
    pickle.dump(usuariopeli_rating, f)

with open('./data/usuariopeli_rating_test.json', 'wb') as f:
    pickle.dump(usuariopeli_rating_test, f)

Hasta aquí las labores de preprocesado realizadas. En resumen lo que hemos hecho es:

* Cargar los datos.
* Tomar una muestra de las películas más votadas y los usuarios más activos.
* Construir cuatro diccionarios, tres para entrenamiento y uno para test.